In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataloader import HangmanTextDataset
import numpy as np
# from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import os
import string
import yaml

In [2]:
# https://github.com/methi1999/hangman/blob/master/model.py 
# take design idea from above repository

### Directed Acyclic Graph for the model

In [3]:
input_word_dim = 28
hidden_dim = 64
input_available_dim = 28 
input_missed_dim = 28 
batch_size = 100
sequence_length = 40
target_dim = 28
num_layers = 2

input_word_tensor = torch.randn(batch_size,sequence_length,input_word_dim)
input_available_tensor = torch.randn(batch_size,input_available_dim)
input_missed_tensor = torch.randn(batch_size, input_missed_dim) 
print("Size of input tensors: \n", input_word_tensor.size(), input_available_tensor.size(), input_missed_tensor.size())

Size of input tensors: 
 torch.Size([100, 40, 28]) torch.Size([100, 28]) torch.Size([100, 28])


In [4]:
lstm_layer = torch.nn.LSTM(input_word_dim, hidden_dim, num_layers,batch_first=True) 
fc_1 = torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim) 
fc_2 = torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim) 
relu = torch.nn.ReLU()

fc_3 = torch.nn.Linear(in_features = input_available_dim, out_features = hidden_dim) 
fc_4 = torch.nn.Linear(in_features = hidden_dim, out_features = hidden_dim) 

fc_5 = torch.nn.Linear(in_features = input_missed_dim, out_features=hidden_dim) 
fc_6 = torch.nn.Linear(in_features = hidden_dim, out_features = hidden_dim)

fc_7 = torch.nn.Linear(in_features= hidden_dim*3, out_features=hidden_dim*2) 
fc_8 = torch.nn.Linear(in_features= hidden_dim*2, out_features=hidden_dim) 
fc_9 = torch.nn.Linear(in_features= hidden_dim, out_features=target_dim) 

soft = torch.nn.Softmax(dim=1)

In [5]:
out, (h_0, c_0) = lstm_layer(input_word_tensor) 
out_lstm = out[:,-1,:]  
print(f"out_lstm({out_lstm.shape})", end = ' --> ')
out_fc_1 = relu(fc_1(out_lstm)) 
print(f"out_fc_1({out_fc_1.shape})", end = ' --> ')
out_fc_2 = relu(fc_2(out_fc_1)) 
print(f"out_fc_2({out_fc_2.shape})", end = ' \n') 

out_fc_3 = relu(fc_3(input_available_tensor)) 
print(f"out_fc_3({out_fc_3.shape})", end = ' --> ')
out_fc_4 = relu(fc_4(out_fc_3)) 
print(f"out_fc_4({out_fc_4.shape})", end = '\n')

out_fc_5 = relu(fc_5(input_missed_tensor)) 
print(f"out_fc_5({out_fc_5.shape})", end = ' --> ')
out_fc_6 = relu(fc_6(out_fc_5)) 
print(f"out_fc_6({out_fc_6.shape})", end = '\n\n')

concat = torch.cat((out_fc_2, out_fc_4, out_fc_5), dim = 1)
print(f"combining outputs from above three networks: ({concat.shape})") 

print("Passing combined features into another feed forward layer")
out = relu(fc_7(concat)) 
print(f"({out.shape})", end = ' --> ')
out = relu(fc_8(out)) 
print(f"({out.shape})", end = ' --> ')
out = relu(fc_9(out)) 
print(f"({out.shape})")

out_lstm(torch.Size([100, 64])) --> out_fc_1(torch.Size([100, 64])) --> out_fc_2(torch.Size([100, 64])) 
out_fc_3(torch.Size([100, 64])) --> out_fc_4(torch.Size([100, 64]))
out_fc_5(torch.Size([100, 64])) --> out_fc_6(torch.Size([100, 64]))

combining outputs from above three networks: (torch.Size([100, 192]))
Passing combined features into another feed forward layer
(torch.Size([100, 128])) --> (torch.Size([100, 64])) --> (torch.Size([100, 28]))


In [6]:
soft(out).detach().numpy().sum()

100.0

### Building a Model 
Including methods like:  
1. \_\_init\_\_()
2. forward()
3. save_model()
4. load_checkpoint() 
5. train_model() 
6. test_model() 
7. predict()

In [12]:
config = {
    "vocab_size": 28,
    "hidden_dim" : 64,
    "input_available_dim" : 28 ,
    "input_missed_dim" : 28 ,
    "batch_size" : 100,
    "max_sequence_length" : 40,
    "target_dim" : 28,
    "num_layers" : 3, 
    "lstm_bidirectional": False, 
    "learning_rate": 1e-2, 
    "use_embedding": False, 
    "embed_dim": 5,
    "models_path": "./checkpoints/"
}

In [13]:
class HangmanSaver(nn.Module):
    """
        Provides basic functionality including load model, save model and inference function. 
        Inference function would be directly called in our guess method that we have to build. 
    """

    def __init__(self, config,desc = "model_trident"):
        super(HangmanSaver, self).__init__()

        self.use_embedding = config["use_embedding"]
        self.models_path = config["models_path"]
        self.desc = desc
        self.seq_length = config['max_sequence_length']
        self.vocab_size = config["vocab_size"]
        self.vocab = ['<pad>'] + list(string.ascii_lowercase) + ['_']
        self.char_to_index = {c:i for i,c in enumerate(self.vocab)}
        self.index_to_char = {i:c for i,c in enumerate(self.vocab)}
        # self.hidden_dim = config["hidden_dim"] 
        # self.input_available_dim = config["input_available_dim"]   
        # self.input_missed_dim = config["input_missed_dim"]   
        # self.batch_size = config["batch_size"] 
        # self.sequence_length = config["sequence_length"] 
        # self.target_dim = config["target_dim"] 
        # self.num_layers = config["num_layers"] 

        self.rec_layer = torch.nn.LSTM(input_size = config["embed_dim"] if config["use_embedding"] else config["vocab_size"], 
                                       hidden_size = config["hidden_dim"], num_layers = config["num_layers"], 
                                       batch_first=True, bidirectional=config["lstm_bidirectional"]) 
        self.fc_1 = torch.nn.Linear(in_features=config["hidden_dim"], out_features=config["hidden_dim"]*4) 
        self.fc_2 = torch.nn.Linear(in_features=config["hidden_dim"]*4, out_features=config["hidden_dim"]) 

        self.fc_3 = torch.nn.Linear(in_features = config["input_available_dim"], out_features = config["hidden_dim"]*2) 
        self.fc_4 = torch.nn.Linear(in_features = config["hidden_dim"]*2, out_features = config["hidden_dim"]) 

        # self.fc_5 = torch.nn.Linear(in_features = config["input_missed_dim"], out_features=config["hidden_dim"]) 
        # self.fc_6 = torch.nn.Linear(in_features = config["hidden_dim"], out_features = config["hidden_dim"])

        self.fc_7 = torch.nn.Linear(in_features= config["hidden_dim"]*2, out_features=config["hidden_dim"]*4) 
        self.fc_8 = torch.nn.Linear(in_features= config["hidden_dim"]*4, out_features=config["hidden_dim"]) 
        self.fc_9 = torch.nn.Linear(in_features= config["hidden_dim"], out_features=config["target_dim"]) 

        self.soft = torch.nn.Softmax(dim=1)
        
        self.relu = torch.nn.ReLU()

        self.optimizer = torch.optim.Adam(self.parameters(), lr = config['learning_rate'])

        self.criterion = torch.nn.BCEWithLogitsLoss()


    def forward(self, x_in, x_available):
        """
        Forward pass through RNN
        :param x_in: input tensor of shape (batch size, max sequence length, input_dim)
        :param x_lens: actual lengths of each sequence < max sequence length (since padded with zeros)
        :param miss_chars: tensor of length batch_size x vocab size. 1 at index i indicates that ith character is NOT present
        :return: tensor of shape (batch size, max sequence length, output dim)
        """        
        if self.use_embedding:
            x_in = self.embedding(x_in)

        out, (h_0, c_0) = self.rec_layer(x_in) 
        out_rec = out[:,-1,:]  
        out_fc_1 = relu(self.fc_1(out_rec)) 
        out_fc_2 = relu(self.fc_2(out_fc_1)) 

        out_fc_3 = relu(self.fc_3(x_available)) 
        out_fc_4 = relu(self.fc_4(out_fc_3)) 

        # out_fc_5 = relu(self.fc_5(x_missed)) 
        # out_fc_6 = relu(self.fc_6(out_fc_5)) 

        concat = torch.cat((out_fc_2, out_fc_4), dim = 1)
        out = relu(self.fc_9(relu(self.fc_8(relu(self.fc_7(concat))))))
        return soft(out)
            

    def loss(self, predicted, truth):
        pass

    # save model, along with loss details and testing accuracy
    # best is the model which has the lowest test loss. This model is used during feature extraction
    def save_model(self, is_best, epoch, train_loss, test_loss):

        base_path = self.models_path
        if is_best:
            filename = base_path + 'best_' + self.desc + '.pth'
        else:
            filename = base_path + "epoch_" + str(epoch) + '_' + self.desc + '.pth'

        torch.save({
            'epoch': epoch,
            'model_state_dict': self.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'train_loss': train_loss,
            'test_loss': test_loss,
        }, filename)

        print("Saved model")

    # Loads saved model for resuming training or inference
    # mode can be 'test' or 'train'
    def load_model(self, checkpoint_path = None):
        # load model parameters and return training/testing loss and testing accuracy
        if checkpoint_path is None: 
            print("Checkpoint path has to be provided...") 
            exit(0)
        checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage)
        self.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Loaded pretrained model from:", checkpoint_path)

        return checkpoint['epoch'], checkpoint['train_loss'], checkpoint['test_loss']
        

    def inference(self,x_in, x_available): 
        """
        Should be the same as forward function but with batch size = 1
        should take input as strings instead of vectors
        should preprocess strings on its own
        should infer possible outcomes on its own.
        """
        def one_hot_encoding(word):
                one_hot = np.zeros((self.seq_length, self.vocab_size))
                for i, ch in enumerate(word):
                    index = self.char_to_index.get(ch)  # Get index from char_to_index dict
                    one_hot[i][index] = 1
                return torch.tensor(one_hot[np.newaxis,:,:], dtype=torch.float32)
            
        # Convert target word to a one-d vector target
        def encode_word_to_vec(word):
            target = np.zeros(self.vocab_size) 
            if type(word)!=str:
                return target
            for ch in word:
                if ch != '_' and ch != '<pad>': 
                    index = self.char_to_index[ch]
                    target[index] = 1
            return torch.tensor(target[np.newaxis,:], dtype = torch.float32) 
        
        x_in = one_hot_encoding(x_in) 
        x_available = encode_word_to_vec(x_available) 

        with torch.no_grad():
            x_pred = self.forward(x_in, x_available)
        word_probs = {w:p for w,p in zip(self.vocab,x_pred.numpy()[0]) if p>0}

        return word_probs

In [14]:
savior = HangmanSaver(config, "untrained")
savior.inference("abcw_t_r","abcdefg")

{'<pad>': 0.034826707,
 'a': 0.034826707,
 'b': 0.036144268,
 'c': 0.034826707,
 'd': 0.034826707,
 'e': 0.03831794,
 'f': 0.038168117,
 'g': 0.034826707,
 'h': 0.036300227,
 'i': 0.034826707,
 'j': 0.036289144,
 'k': 0.034826707,
 'l': 0.035730794,
 'm': 0.034826707,
 'n': 0.034826707,
 'o': 0.034826707,
 'p': 0.034826707,
 'q': 0.034826707,
 'r': 0.039371286,
 's': 0.03881577,
 't': 0.037910808,
 'u': 0.034826707,
 'v': 0.034826707,
 'w': 0.034826707,
 'x': 0.034826707,
 'y': 0.034826707,
 'z': 0.036070973,
 '_': 0.034826707}

In [15]:
# for batch_input, batch_available, batch_missed, batch_target in tqdm(dataloader):
#     batch_input = torch.squeeze(batch_input) 
#     batch_available = torch.squeeze(batch_available) 
#     batch_missed = torch.squeeze(batch_missed)
#     batch_target = torch.squeeze(batch_target)
#     print(batch_input.shape, batch_available.shape, batch_missed.shape, batch_target.shape)
#     break

In [23]:
model = HangmanSaver(config, "training_pipeline_testing") 
# Load dataset and create DataLoader
dataset = HangmanTextDataset('./data/train/train_mini.csv', chunk_size=1000)
dataloader = DataLoader(dataset)


# Training loop
import os
resume = True
global epoch
total_epochs = 100
if os.path.exists('./checkpoint.pth') and resume: 
    checkpoint = torch.load('./checkpoint.pth')  
    model.load_model('./checkpoint.pth') 
    epoch = checkpoint['epoch'] + 1
    loss = checkpoint['loss'] 
else: 
    epoch = 0



model.train()
while epoch < total_epochs:
    running_loss = 0.0
    for batch_input, batch_available, batch_target in tqdm(dataloader, desc=f"Epoch {epoch+1}/{total_epochs}"):
        batch_input = torch.squeeze(batch_input) 
        batch_available = torch.squeeze(batch_available) 
        batch_target = torch.squeeze(batch_target)
        
        # Forward pass
        outputs = model(batch_input, batch_available)

        # Compute loss
        loss = model.criterion(outputs, batch_target)

        # Backward pass and optimization
        model.optimizer.zero_grad()
        loss.backward()
        model.optimizer.step()

        running_loss += loss.item() * batch_input.size(0)

    epoch_loss = running_loss / 4120
    print(f"Epoch {epoch+1}/{total_epochs}, Loss: {epoch_loss:.8f}")
    # writer.add_scalar('Loss/train', epoch_loss, epoch)

    # Save checkpoint
    print(model.inference("__ttlebo_e","acdfhijknpqruvwxyz")) #target cenu : cuttlebone
    model.save_model(is_best=True, epoch=epoch, train_loss = epoch_loss, test_loss = 10)
    # torch.save({
    #     'epoch': epoch,
    #     'model_state_dict': model.state_dict(),
    #     'optimizer_state_dict': model.optimizer.state_dict(),
    #     'loss': loss,
    # }, checkpoint_path) 

    epoch += 1

# Close Tensorboard writer
# writer.close()


Epoch 1/100: 0it [00:00, ?it/s]

Epoch 1/100: 5it [00:02,  1.72it/s]


Epoch 1/100, Loss: 0.70275743
{'<pad>': 2.670847e-11, 'a': 0.8640806, 'b': 2.670847e-11, 'c': 2.670847e-11, 'd': 2.670847e-11, 'e': 0.12519504, 'f': 2.670847e-11, 'g': 2.670847e-11, 'h': 2.670847e-11, 'i': 2.670847e-11, 'j': 2.670847e-11, 'k': 2.670847e-11, 'l': 1.0561412e-06, 'm': 2.670847e-11, 'n': 0.0073724072, 'o': 2.670847e-11, 'p': 4.882706e-11, 'q': 2.670847e-11, 'r': 3.7424303e-05, 's': 0.0033135551, 't': 2.670847e-11, 'u': 2.670847e-11, 'v': 2.670847e-11, 'w': 2.670847e-11, 'x': 2.670847e-11, 'y': 2.670847e-11, 'z': 2.670847e-11, '_': 2.670847e-11}
Saved model


Epoch 2/100: 5it [00:02,  1.80it/s]


Epoch 2/100, Loss: 0.70075073
{'<pad>': 3.6617035e-05, 'a': 0.0011246795, 'b': 3.6617035e-05, 'c': 0.00013129057, 'd': 3.6617035e-05, 'e': 0.7226538, 'f': 3.6617035e-05, 'g': 3.6617035e-05, 'h': 3.6617035e-05, 'i': 3.6617035e-05, 'j': 3.6617035e-05, 'k': 3.6617035e-05, 'l': 0.0046695336, 'm': 3.6617035e-05, 'n': 0.09789756, 'o': 3.6617035e-05, 'p': 3.6617035e-05, 'q': 3.6617035e-05, 'r': 0.101768814, 's': 0.07098537, 't': 3.6617035e-05, 'u': 3.6617035e-05, 'v': 3.6617035e-05, 'w': 3.6617035e-05, 'x': 3.6617035e-05, 'y': 3.6617035e-05, 'z': 3.6617035e-05, '_': 3.6617035e-05}
Saved model


...

Epoch 52/100, Loss: 0.69356409
{'<pad>': 1.0173659e-05, 'a': 0.8061943, 'b': 1.0173659e-05, 'c': 1.0173659e-05, 'd': 1.0173659e-05, 'e': 1.0173659e-05, 'f': 1.0173659e-05, 'g': 1.0173659e-05, 'h': 1.0173659e-05, 'i': 0.030622672, 'j': 1.0173659e-05, 'k': 1.0173659e-05, 'l': 6.004975e-05, 'm': 1.0173659e-05, 'n': 0.16183844, 'o': 1.0173659e-05, 'p': 1.0173659e-05, 'q': 1.0173659e-05, 'r': 0.00010754081, 's': 0.00095305685, 't': 1.0173659e-05, 'u': 1.0173659e-05, 'v': 1.0173659e-05, 'w': 1.0173659e-05, 'x': 1.0173659e-05, 'y': 1.0173659e-05, 'z': 1.0173659e-05, '_': 1.0173659e-05}
Saved model


Epoch 53/100: 5it [00:07,  1.51s/it]


Epoch 53/100, Loss: 0.69133574
{'<pad>': 3.2574884e-05, 'a': 0.96142036, 'b': 3.2574884e-05, 'c': 3.2574884e-05, 'd': 3.2574884e-05, 'e': 3.2574884e-05, 'f': 3.2574884e-05, 'g': 3.2574884e-05, 'h': 3.2574884e-05, 'i': 0.020453254, 'j': 3.2574884e-05, 'k': 3.2574884e-05, 'l': 0.00018211869, 'm': 3.2574884e-05, 'n': 0.0161374, 'o': 3.2574884e-05, 'p': 3.2574884e-05, 'q': 3.2574884e-05, 'r': 0.00020282116, 's': 0.0008874351, 't': 3.2574884e-05, 'u': 3.2574884e-05, 'v': 3.2574884e-05, 'w': 3.2574884e-05, 'x': 3.2574884e-05, 'y': 3.2574884e-05, 'z': 3.2574884e-05, '_': 3.2574884e-05}
Saved model


Epoch 54/100: 1it [00:06,  6.37s/it]


KeyboardInterrupt: 

In [22]:
sum(p.numel() for p in model.parameters())
# you should have around 10 times more data samples than parameters.

186972

In [22]:
model.inference("_at_r","bcdefghimnopqsuvwxyz","jkl")

{}

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not str

In [4]:

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size//2)
        self.fc2 = nn.Linear(hidden_size//2, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.relu(lstm_out[:, -1, :])
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [21]:
# Parameters
input_size = 28
hidden_size = 64
output_size = 28
seq_length = 10
batch_size = 32
num_epochs = 15
num_layers = 2
learning_rate = 0.001
checkpoint_path = 'checkpoint.pth'

# Instantiate the model, loss function, and optimizer
model = LSTMClassifier(input_size, hidden_size,num_layers, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Load dataset and create DataLoader
dataset = HangmanTextDataset('./data/train/mini.csv', chunk_size=1000)
dataloader = DataLoader(dataset)

In [59]:
# Tensorboard writer for logging
# writer = SummaryWriter()

# Training loop
import os
resume = False
global epoch
if os.path.exists('./checkpoint.pth') and resume: 
    model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

    checkpoint = torch.load('./checkpoint.pth')  
    model.load_state_dict(checkpoint['model_state_dict']) 
    optimizer.load_state_dict(checkpoint['optimizer_state_dict']) 
    epoch = checkpoint['epoch'] + 1
    loss = checkpoint['loss'] 
else: 
    epoch = 0


while epoch < num_epochs:
    model.train()
    running_loss = 0.0
    for batch_input, batch_target in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_input = torch.squeeze(batch_input) 
        batch_target = torch.squeeze(batch_target)
        # Forward pass
        outputs = model(batch_input)

        # Compute loss
        loss = criterion(outputs, batch_target)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_input.size(0)

    epoch_loss = running_loss / 40000
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    # writer.add_scalar('Loss/train', epoch_loss, epoch)

    # Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
    }, checkpoint_path) 

    epoch += 1

# Close Tensorboard writer
# writer.close()


NameError: name 'num_epochs' is not defined